In [17]:
import xarray as xr
import numpy as np
import json
from tqdm import notebook

# Load WRF NetCDF file
ds = xr.open_dataset("/Users/manaruchi/Desktop/WeatherDataViz/raw_data/AFCNWP_WRF_model_output_00UTC.nc")

tlimit = 72 # hourly data for 3 days (24x3)

for t in notebook.tqdm(range(tlimit), desc="Generating Hourly Wind JSON"):
    
    # Extract U10 and V10 (10 m winds)
    u10 = ds["U10"].isel(Time=t)
    v10 = ds["V10"].isel(Time=t)
    lat = ds["XLAT"].isel(Time=t)
    lon = ds["XLONG"].isel(Time=t)
    time_string = str(ds["XTIME"].isel(Time=t).values)
    
    # Convert from m/s to knots
    u10_knots = u10.values * 1.94384
    v10_knots = v10.values * 1.94384
    
    ny, nx = u10.shape

    # Shared header metadata
    base_header = {
        "parameterUnit": "KT",
        "parameterCategory": 2,
        "nx": nx,
        "ny": ny,
        "lo1": float(lon[0, 0]),
        "la1": float(lat[0, 0]),
        "lo2": float(lon[-1, -1]),
        "la2": float(lat[-1, -1]),
        "dx": float((lon[0, -1] - lon[0, 0]) / (nx - 1)),
        "dy": float((lat[0, 0] - lat[-1, 0]) / (ny - 1)),
        "refTime": f'{time_string.split("T")[0]} {time_string.split("T")[1].split(":")[0]}:00:00',
        "forecastTime": 0
    }
    
    # Create two objects: eastward and northward winds
    eastward = {
        "header": {**base_header, "parameterNumber": 2, "parameterNumberName": "eastward_wind"},
        "data": u10_knots.flatten().tolist()
    }
    
    northward = {
        "header": {**base_header, "parameterNumber": 3, "parameterNumberName": "northward_wind"},
        "data": v10_knots.flatten().tolist()
    }
    
    # Combine into one array (expected by Leaflet Velocity)
    combined = [eastward, northward]

    
    
    # Write to file
    with open(f"/Users/manaruchi/Desktop/WeatherDataViz/test/data/wind_data/wind_Surface_{t}.json", "w") as f:
        json.dump(combined, f)
    


Generating Hourly Wind JSON:   0%|          | 0/72 [00:00<?, ?it/s]

# Generate Wind File for 300hPa

In WRF (Weather Research and Forecasting model) output, the full model pressure at each grid point is not directly stored as a single variable — instead, it's typically split into two parts:

✅ The Two Components of Pressure in WRF
PB — Base state pressure (Pa)
This is the hydrostatic reference pressure field that is constant in time.

P — Perturbation pressure (Pa)
This is the deviation from the base state due to weather dynamics at each time and grid point.

✅ Full Pressure Formula
The full pressure at each grid point is:

Full Pressure (Pa) = P + PB

If you want pressure in hPa, divide by 100:

Pressure (hPa) = P + PB / 100

## Calculation Part

Lets find out the pressure values at each vertical level. Our model has 32 levels.

In [9]:
avg_pressure = ((ds["P"] + ds["PB"]) / 100).mean(dim=["Time", "south_north", "west_east"])

for i, v in enumerate(avg_pressure.values):
    print(i+1,v)

1 925.67596
2 919.47375
3 911.6386
4 901.81964
5 889.63293
6 874.685
7 856.61194
8 835.12115
9 810.0484
10 781.4113
11 749.4497
12 714.624
13 677.5836
14 639.00775
15 599.2603
16 558.45807
17 516.7484
18 474.3227
19 431.42194
20 388.34344
21 345.44986
22 303.16223
23 261.95807
24 222.5446
25 187.03839
26 156.94997
27 131.70135
28 110.51484
29 92.73604
30 77.81701
31 65.29808
32 54.793236


## Calculate Winds for 300hPa

Now level 22 is close to 300hPa. So we can use this level data. Syntax would be:-

`u = ds["U"].isel(Time=t, bottom_top=22)`

In [7]:
len(avg_pressure)

32

In [15]:
import xarray as xr
import numpy as np
import json
from tqdm import notebook

# Load WRF NetCDF file
ds = xr.open_dataset("/Users/manaruchi/Desktop/WeatherDataViz/raw_data/AFCNWP_WRF_model_output_00UTC.nc")

tlimit = 72 # hourly data for 3 days (24x3)

for t in notebook.tqdm(range(tlimit), desc="Generating Hourly Wind JSON"):
    
    # Extract U and V
    u = ds["U"].isel(Time=t, bottom_top=22)
    v = ds["V"].isel(Time=t, bottom_top=22)
    lat = ds["XLAT"].isel(Time=t)
    lon = ds["XLONG"].isel(Time=t)
    time_string = str(ds["XTIME"].isel(Time=t).values)
    
    # Convert from m/s to knots
    u_knots = u.values * 1.94384
    v_knots = v.values * 1.94384
    
    ny, nx = u.shape

    # Shared header metadata
    base_header = {
        "parameterUnit": "KT",
        "parameterCategory": 2,
        "nx": nx,
        "ny": ny,
        "lo1": float(lon[0, 0]),
        "la1": float(lat[0, 0]),
        "lo2": float(lon[-1, -1]),
        "la2": float(lat[-1, -1]),
        "dx": float((lon[0, -1] - lon[0, 0]) / (nx - 1)),
        "dy": float((lat[0, 0] - lat[-1, 0]) / (ny - 1)),
        "refTime": f'{time_string.split("T")[0]} {time_string.split("T")[1].split(":")[0]}:00:00',
        "forecastTime": 0
    }
    
    # Create two objects: eastward and northward winds
    eastward = {
        "header": {**base_header, "parameterNumber": 2, "parameterNumberName": "eastward_wind"},
        "data": u_knots.flatten().tolist()
    }
    
    northward = {
        "header": {**base_header, "parameterNumber": 3, "parameterNumberName": "northward_wind"},
        "data": v_knots.flatten().tolist()
    }
    
    # Combine into one array (expected by Leaflet Velocity)
    combined = [eastward, northward]

    
    
    # Write to file
    with open(f"/Users/manaruchi/Desktop/WeatherDataViz/test/data/wind_data/wind_test.json", "w") as f:
        json.dump(combined, f)

    import sys
    sys.exit()
    


Generating Hourly Wind JSON:   0%|          | 0/72 [00:00<?, ?it/s]

SystemExit: 

/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Full Wind Data Extraction Code

In [ ]:
import xarray as xr
import numpy as np
import json
from tqdm import notebook

# Load WRF NetCDF file
ds = xr.open_dataset("/Users/manaruchi/Desktop/WeatherDataViz/raw_data/AFCNWP_WRF_model_output_00UTC.nc")

tlimit = 72 # hourly data for 3 days (24x3)

for t in notebook.tqdm(range(tlimit), desc="Generating Hourly Wind JSON"):
    
    # Extract U and V
    u = ds["U"].isel(Time=t, bottom_top=22)
    v = ds["V"].isel(Time=t, bottom_top=22)
    lat = ds["XLAT"].isel(Time=t)
    lon = ds["XLONG"].isel(Time=t)
    time_string = str(ds["XTIME"].isel(Time=t).values)
    
    # Convert from m/s to knots
    u_knots = u.values * 1.94384
    v_knots = v.values * 1.94384
    
    ny, nx = u.shape

    # Shared header metadata
    base_header = {
        "parameterUnit": "KT",
        "parameterCategory": 2,
        "nx": nx,
        "ny": ny,
        "lo1": float(lon[0, 0]),
        "la1": float(lat[0, 0]),
        "lo2": float(lon[-1, -1]),
        "la2": float(lat[-1, -1]),
        "dx": float((lon[0, -1] - lon[0, 0]) / (nx - 1)),
        "dy": float((lat[0, 0] - lat[-1, 0]) / (ny - 1)),
        "refTime": f'{time_string.split("T")[0]} {time_string.split("T")[1].split(":")[0]}:00:00',
        "forecastTime": 0
    }
    
    # Create two objects: eastward and northward winds
    eastward = {
        "header": {**base_header, "parameterNumber": 2, "parameterNumberName": "eastward_wind"},
        "data": u_knots.flatten().tolist()
    }
    
    northward = {
        "header": {**base_header, "parameterNumber": 3, "parameterNumberName": "northward_wind"},
        "data": v_knots.flatten().tolist()
    }
    
    # Combine into one array (expected by Leaflet Velocity)
    combined = [eastward, northward]

    
    
    # Write to file
    with open(f"/Users/manaruchi/Desktop/WeatherDataViz/test/data/wind_data/wind_test.json", "w") as f:
        json.dump(combined, f)

    import sys
    sys.exit()